In [92]:
import pandas as pd
import numpy as np
import os
import csv
import re

from os import walk

## Make a csv with file names

In [ ]:
with open("headpose_names.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['directory', 'filename'])
        for root, dirs, files in os.walk(root_path):
            for file in files:
                if file.lower().endswith('.jpg'):
                    filename = file
                    writer.writerow([root.split('/')[-1], filename])

## Manually decode direction

In [93]:
df = pd.read_csv("headpose_names.csv")
df.head()

,directory,filename
0,Person05,person05286+60+15.jpg
1,Person05,person05142+0-60.jpg
2,Person05,person05287+60+30.jpg
3,Person05,person05146+0+0.jpg
4,Person05,person05124-30+60.jpg


In [78]:
row = df.iloc[0]
rotation = re.findall(r"[-+]?\d+", row['filename'])[1:]
print(row)
x_rotation = int(rotation[-1])
y_rotation = int(rotation[-2])
x_rotation

directory                 Person05
filename     person05286+60+15.jpg
Name: 0, dtype: object


15

In [88]:
def calculate_direction(row):
    """ Only basic directions """
    rotation = re.findall(r"[-+]?\d+", row['filename'])[1:]
    x_rotation = int(rotation[-1])
    y_rotation = int(rotation[-2])

    if abs(y_rotation) >= abs(x_rotation):
        if y_rotation > 0:
            return 'Up'
        elif y_rotation < 0:
            return 'Down'
        else:
            return 'Front'
    else:
        if x_rotation > 0:
            return 'Right'
        elif x_rotation < 0:
            return 'Left'
        else:
            return 'Front'


In [89]:
df['direction'] = df.apply(calculate_direction, axis=1)

direction_mapping = {'Front': 0, 'Up': 1, 'Down': 2, 'Right': 3, 'Left': 4}
df['direction'] = df['direction'].map(direction_mapping)

In [90]:
df.head()

,directory,filename,direction
0,Person05,person05286+60+15.jpg,1
1,Person05,person05142+0-60.jpg,4
2,Person05,person05287+60+30.jpg,1
3,Person05,person05146+0+0.jpg,0
4,Person05,person05124-30+60.jpg,3


In [91]:
df.to_csv("headpose_directions_simple.csv", index=False)

In [94]:
def calculate_direction(row):
    """ Also with diagonal directions """
    rotation = re.findall(r"[-+]?\d+", row['filename'])[1:]
    x_rotation = int(rotation[-1])
    y_rotation = int(rotation[-2])

    if abs(y_rotation) >= abs(x_rotation):
        if y_rotation > 0:
            if x_rotation > 0:
                return 'Up-Right'
            elif x_rotation < 0:
                return 'Up-Left'
            else:
                return 'Up'
        elif y_rotation < 0:
            if x_rotation > 0:
                return 'Down-Right'
            elif x_rotation < 0:
                return 'Down-Left'
            else:
                return 'Down'
        else:
            return 'Front'
    else:
        if x_rotation > 0:
            return 'Right'
        elif x_rotation < 0:
            return 'Left'
        else:
            return 'Front'


In [95]:
df['direction'] = df.apply(calculate_direction, axis=1)

direction_mapping = {
    'Front': 0,
    'Up': 1,
    'Down': 2,
    'Right': 3,
    'Left': 4,
    'Up-Right': 5,
    'Up-Left': 6,
    'Down-Right': 7,
    'Down-Left': 8
}
df['direction'] = df['direction'].map(direction_mapping)

In [96]:
df.head()

,directory,filename,direction
0,Person05,person05286+60+15.jpg,5
1,Person05,person05142+0-60.jpg,4
2,Person05,person05287+60+30.jpg,5
3,Person05,person05146+0+0.jpg,0
4,Person05,person05124-30+60.jpg,3


In [97]:
df.to_csv("headpose_directions_complex.csv", index=False)

## Merge csvs with direction and pitch, yaw, roll values

In [102]:
df_simple = pd.read_csv("headpose_directions_simple.csv")
df_simple.head()

,directory,filename,direction
0,Person05,person05286+60+15.jpg,1
1,Person05,person05142+0-60.jpg,4
2,Person05,person05287+60+30.jpg,1
3,Person05,person05146+0+0.jpg,0
4,Person05,person05124-30+60.jpg,3


In [103]:
df_pyr = pd.read_csv("headpose.csv", names=['directory', 'filename', 'pitch', 'yaw', 'roll'])
df_pyr.head()

,directory,filename,pitch,yaw,roll
0,Front,personne09146+0+0.jpg,-0.016522,0.150272,0.004123
1,Front,personne06246+0+0.jpg,0.030302,0.042676,-0.183583
2,Front,personne01146+0+0.jpg,0.017203,-0.081454,-0.017137
3,Front,personne06146+0+0.jpg,0.069600,0.142844,-0.146851
4,Front,personne07146+0+0.jpg,0.047875,-0.220413,0.048016


In [105]:
merged = df_pyr.merge(df_simple, on=['filename', 'directory'])
merged.head()

,directory,filename,pitch,yaw,roll,direction
0,Front,personne09146+0+0.jpg,-0.016522,0.150272,0.004123,0
1,Front,personne06246+0+0.jpg,0.030302,0.042676,-0.183583,0
2,Front,personne01146+0+0.jpg,0.017203,-0.081454,-0.017137,0
3,Front,personne06146+0+0.jpg,0.069600,0.142844,-0.146851,0
4,Front,personne07146+0+0.jpg,0.047875,-0.220413,0.048016,0


In [106]:
merged

,directory,filename,pitch,yaw,roll,direction
0,Front,personne09146+0+0.jpg,-0.016522,0.150272,0.004123,0
1,Front,personne06246+0+0.jpg,0.030302,0.042676,-0.183583,0
2,Front,personne01146+0+0.jpg,0.017203,-0.081454,-0.017137,0
3,Front,personne06146+0+0.jpg,0.069600,0.142844,-0.146851,0
4,Front,personne07146+0+0.jpg,0.047875,-0.220413,0.048016,0
...,...,...,...,...,...,...
2811,Person15,person15110-60+45.jpg,-0.843629,0.905632,0.675885,2
2812,Person15,person15123-30+45.jpg,-0.684637,0.738273,0.824305,3
2813,Person15,person15207-60+0.jpg,0.029584,0.051617,0.734259,2
2814,Person15,person15288+60+45.jpg,0.198533,0.606636,-0.622640,1


In [107]:
merged.to_csv("headpose_merged.csv", index=False)